In [5]:

# coding: utf-8

# In[ ]:


from __future__ import print_function

import sys, os
import numpy as np
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
os.environ['XILINX_XRT'] = '/usr'
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))
    print("Start of \"" + sys.argv[0] + "\"")
    
    # Overlay
    ol = Overlay("iFFT.bit")
    ipiFFT = ol.iFFT_0

    # Memory allocate
    inBuffer0 = allocate(shape=(1024,), dtype=np.float64)
    
    sum = 0
    for i in range(10):
        iFFT_in = open("iFFT_in.txt", "r+")
        iFFT_in.seek(0)
        for i in range(1024):
            line = iFFT_in.readline()
            inBuffer0[i] = float(line)
        iFFT_in.close()
        timeKernelStart = time()
    # ============= Set Kernel Argument ==============
    # control
    # 0x00 : Control signals
    #        bit 0  - ap_start (Read/Write/COH)
    #        bit 1  - ap_done (Read/COR)
    #        bit 2  - ap_idle (Read)
    # 0x10 : Data signal of f
    #        bit 31~0 - f[31:0] (Read/Write)
    # 0x14 : Data signal of f
    #        bit 31~0 - f[63:32] (Read/Write)
    # 0x1c : Data signal of logn
    #        bit 31~0 - logn[31:0] (Read/Write)
        logn = 10
        ipiFFT.write(0x1C, logn) # Coefficient

    # ================ Data Transfer =================
        ipiFFT.write(0x10, inBuffer0.device_address) # 0x10 bit 31:0
        ipiFFT.write(0x14, inBuffer0.device_address) # 0x14 bit 63:32

    # =================== Start ======================
        ipiFFT.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
        while (ipiFFT.read(0x00) & 0x4) == 0x0:
            continue
        timeKernelEnd = time()
        print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
        sum += timeKernelEnd - timeKernelStart
    
    # ================== Compare =====================
        error_cnt = 0
        iFFT_out = open("iFFT_out.txt", "r+")
        iFFT_out.seek(0)
        for i in range(1024):
            line = iFFT_out.readline()
            golden = round(float(line), 5)  # round to the fifth decimal place
            actual = round(inBuffer0[i], 5) # round to the fifth decimal place
            if actual != golden:
                print(f"Failed, expect value: {golden:.5f}, actual value: {actual:.5f}")
                error_cnt += 1
            #else:
                #print(f"Calculate value: {inBuffer0[i]:.17f}")
        iFFT_out.close()
        if error_cnt == 0:
            print("Test Passed!!")
        else:
            print("Test Failed!!")
    print("Average time: ", sum/10)
    print("============================")
    print("Exit process")



Entry: /usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py"
Kernel execution time: 0.00266265869140625 s
Test Passed!!
Kernel execution time: 0.0026302337646484375 s
Test Passed!!
Kernel execution time: 0.0026221275329589844 s
Test Passed!!
Kernel execution time: 0.0026216506958007812 s
Test Passed!!
Kernel execution time: 0.0026319026947021484 s
Test Passed!!
Kernel execution time: 0.0026433467864990234 s
Test Passed!!
Kernel execution time: 0.0026488304138183594 s
Test Passed!!
Kernel execution time: 0.002626657485961914 s
Test Passed!!
Kernel execution time: 0.0026297569274902344 s
Test Passed!!
Kernel execution time: 0.0026171207427978516 s
Test Passed!!
Average time:  0.0026334285736083984
Exit process
